In [ ]:
import energy_consumption as ec
import os
import sys
current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, "../.."))
sys.path.append(project_root)

import simulation.display_results as dr
import json
import math
import numpy as np
import json
from pprint import pprint

In [ ]:
# Define vehicle characteristics
with open("vehicle_data.json", "r") as file:
    vehicle_data = json.load(file)
with open("battery_data.json", "r") as file:
    battery_data = json.load(file)
# Define static data
static_data = {
    "air_dens": 1.225,  # kg/m³
    "grav_acc": 9.81,  # m/s²
}

# Define route segments
segments = [
    {
        "segment_name": "Flat start",
        "points": ["point1", "point2"],
        "coords": [-2.6080503, 51.4562142, -2.6080419, 51.4562768],
        "climb": 0,
        "distance": 50,
        "avg_incline_angle": 0,
        "stop_start": True,  # Starting from a stop
    },
    {
        "segment_name": "Gentle uphill",
        "points": ["point2", "point3"],
        "coords": [-2.6080419, 51.4562768, -2.6080335, 51.4563394],
        "climb": 1.5,
        "distance": 70,
        "avg_incline_angle": 2.5,
        "stop_start": False,  # Continuing from previous segment
    },
    {
        "segment_name": "Steep uphill",
        "points": ["point3", "point4"],
        "coords": [-2.6080335, 51.4563394, -2.6080251, 51.4564020],
        "climb": 4,
        "distance": 60,
        "avg_incline_angle": 6.8,
        "stop_start": False,  # Continuing from previous segment
    },
    {
        "segment_name": "Traffic light stop",
        "points": ["point4", "point5"],
        "coords": [-2.6080251, 51.4564020, -2.6080167, 51.4564646],
        "climb": 0,
        "distance": 80,
        "avg_incline_angle": 0,
        "stop_start": True,  # Stopped at traffic light
    },
    {
        "segment_name": "Downhill",
        "points": ["point5", "point6"],
        "coords": [-2.6080167, 51.4564646, -2.6080083, 51.4565272],
        "climb": -3,
        "distance": 100,
        "avg_incline_angle": -3.0,
        "stop_start": False,  # Continuing from previous segment
    }
]


# E-bike specifications
target_velocity = 5.5  # m/s (approx. 20 km/h)
max_motor_power = vehicle_data['max_motor_power']  # Watts (EU legal limit)
motor_efficiency = 0.85  # 85% efficiency


In [ ]:
# Process the route
results = ec.process_route_segments(
    segments, vehicle_data, static_data, 
    target_velocity, max_motor_power, motor_efficiency, battery_data["OCV"], battery_data["R_internal"], battery_data["Capacity"]
)
pprint(results)
dr.display_route_results(results, segments)